In [ ]:
# Cell 1: Imports & Setup for SOL Supercomputer (Headless Mode)
# solAutoencoder3: Enhanced SimpleUNet + 256x256 patches + Overlapping tiles + Nodata masking

import numpy as np
import rasterio
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

# Files
l1_file = "satellite_data/LC08_L1TP_037037_20250603_20250607_02_T1_combined.tif"
l2_file = "satellite_data/LC08_L2SP_037037_20250603_20250607_02_T1_combined.tif"

L2_THERMAL_BAND_INDEX = 6  

# CONFIG - Optimized for accuracy
PATCH_SIZE = 256       # Smaller patches = more training samples
TILE_STRIDE = 128      # 50% overlap during tiling = 4x more samples
BATCH_SIZE = 16        
EPOCHS = 150           
NUM_WORKERS = 8        
LR = 1e-4              

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# Cell 2: Load Data & Normalize

with rasterio.open(l1_file) as src1:
    L1_all = src1.read().astype(np.float32)
    l1_profile = src1.profile

with rasterio.open(l2_file) as src2:
    L2 = src2.read(L2_THERMAL_BAND_INDEX).astype(np.float32)
    l2_profile = src2.profile

# Create nodata mask (IMPORTANT: will be used during inference)
nodata_mask = (L2 > 0)
print(f"Valid pixels: {np.sum(nodata_mask):,} / {nodata_mask.size:,} ({100*np.sum(nodata_mask)/nodata_mask.size:.1f}%)")

# Normalize L1
L1_norm = np.empty_like(L1_all, dtype=np.float32)
for b in range(L1_all.shape[0]):
    band = L1_all[b]
    mask = (band > 0) & nodata_mask
    p1, p99 = np.percentile(band[mask], [1, 99])
    L1_norm[b] = np.clip((band - p1) / (p99 - p1), 0, 1)

# Normalize L2
L2_masked = L2[nodata_mask]
L2_p1, L2_p99 = np.percentile(L2_masked, [1, 99])
L2_norm = np.clip((L2 - L2_p1) / (L2_p99 - L2_p1), 0, 1)

print("Data loaded and normalized.")
print(f"L2 normalization range: {L2_p1:.2f} K to {L2_p99:.2f} K")


In [ ]:
# Cell 3: Overlapping Tiling (256x256 with 128 stride = 4x more samples)

def tile_multiband_overlapping(img_3d, target_2d, valid_mask_2d, patch=PATCH_SIZE, stride=TILE_STRIDE):
    """
    Create overlapping tiles for training.
    With stride = patch/2, each pixel appears in up to 4 different tiles.
    """
    C, H, W = img_3d.shape
    tiles_x, tiles_y = [], []
    
    for i in range(0, H - patch + 1, stride):
        for j in range(0, W - patch + 1, stride):
            x_patch = img_3d[:, i:i+patch, j:j+patch]
            y_patch = target_2d[i:i+patch, j:j+patch]
            mask_patch = valid_mask_2d[i:i+patch, j:j+patch]
            
            # Require at least 50% valid pixels (stricter than before)
            valid_ratio = np.mean(mask_patch)
            if valid_ratio < 0.5:
                continue
            
            tiles_x.append(x_patch)
            tiles_y.append(y_patch)
    
    X = np.stack(tiles_x, axis=0)
    Y = np.stack(tiles_y, axis=0)[:, None, ...]
    return X, Y

X_tiles, Y_tiles = tile_multiband_overlapping(L1_norm, L2_norm, nodata_mask, PATCH_SIZE, TILE_STRIDE)
print(f"Generated {X_tiles.shape[0]} overlapping tiles of size {PATCH_SIZE}x{PATCH_SIZE}")
print(f"(Compare: non-overlapping would give ~{(7891//PATCH_SIZE)*(7751//PATCH_SIZE)} tiles)")


In [ ]:
# Cell 4: Dataset with Augmentation

class LandsatMultiBandDataset(Dataset):
    def __init__(self, X, Y, augment=False):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()
        self.augment = augment

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x, y = self.X[idx], self.Y[idx]

        if self.augment:
            # Random horizontal flip
            if torch.rand(1) > 0.5:
                x = torch.flip(x, [-1])
                y = torch.flip(y, [-1])
            # Random vertical flip
            if torch.rand(1) > 0.5:
                x = torch.flip(x, [-2])
                y = torch.flip(y, [-2])
            # Random rotation
            k = torch.randint(0, 4, (1,)).item()
            if k > 0:
                x = torch.rot90(x, k, [-2, -1])
                y = torch.rot90(y, k, [-2, -1])

        return x, y

full_indices = np.arange(len(X_tiles))
np.random.seed(42)
train_indices = np.random.choice(full_indices, size=int(0.8 * len(full_indices)), replace=False)
val_indices = np.setdiff1d(full_indices, train_indices)

train_ds = LandsatMultiBandDataset(X_tiles[train_indices], Y_tiles[train_indices], augment=True)
val_ds   = LandsatMultiBandDataset(X_tiles[val_indices], Y_tiles[val_indices], augment=False)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f"Train tiles: {len(train_ds)}, Val tiles: {len(val_ds)}")


In [ ]:
# Cell 5: Enhanced SimpleUNet Model (Wider + Deeper)

class EnhancedSimpleUNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=1):
        super().__init__()
        
        # Encoder (wider filters: 32, 64, 128, 256)
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.pool1 = nn.MaxPool2d(2)
        
        self.enc2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.pool2 = nn.MaxPool2d(2)
        
        self.enc3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.pool3 = nn.MaxPool2d(2)
        
        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        
        # Decoder
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        
        self.up1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(64, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, out_channels, 3, padding=1),
        )

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        
        e3 = self.enc3(p2)
        p3 = self.pool3(e3)
        
        b = self.bottleneck(p3)
        
        u3 = self.up3(b)
        c3 = torch.cat([u3, e3], dim=1)
        d3 = self.dec3(c3)
        
        u2 = self.up2(d3)
        c2 = torch.cat([u2, e2], dim=1)
        d2 = self.dec2(c2)
        
        u1 = self.up1(d2)
        c1 = torch.cat([u1, e1], dim=1)
        out = self.dec1(c1)
        
        return out

model = EnhancedSimpleUNet(in_channels=6, out_channels=1).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model initialized: EnhancedSimpleUNet")
print(f"Total parameters: {total_params:,} (~{total_params/1e6:.2f}M)")

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
criterion = nn.L1Loss()


In [ ]:
# Cell 6: Training Loop

best_val_loss = float('inf')
history = {'train_loss': [], 'val_loss': []}
start_time = time.time()

print(f"Starting training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    for i, (x, y) in enumerate(train_dl):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_dl)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            val_loss += criterion(pred, y).item()
            
    val_loss /= len(val_dl)
    
    scheduler.step(val_loss)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    print(f"Epoch {epoch+1}/{EPOCHS}: Train Loss={train_loss:.6f}, Val Loss={val_loss:.6f} (Time: {time.time()-start_time:.1f}s)")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_sol3_model.pth")
        print(f"  >>> Saved new best model (Val Loss: {val_loss:.6f})")

print(f"Training complete. Total time: {time.time()-start_time:.1f}s")


In [ ]:
# Cell 7: Save Learning Curve Plot
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('L1 Loss')
plt.title('Training History (EnhancedSimpleUNet + Overlapping Tiles)')
plt.legend()
plt.grid(True)
plt.savefig("sol3_training_history.png") 
print("Saved learning curve to sol3_training_history.png")


In [ ]:
# Cell 8: Overlapping Inference WITH Nodata Masking

model.load_state_dict(torch.load("best_sol3_model.pth"))
model.eval()

C, H, W = L1_norm.shape
stride = PATCH_SIZE // 2  # 50% overlap for smooth predictions

pred_sum = np.zeros((H, W), dtype=np.float32)
pred_count = np.zeros((H, W), dtype=np.float32)

n_rows = int(np.ceil((H - PATCH_SIZE) / stride) + 1)
n_cols = int(np.ceil((W - PATCH_SIZE) / stride) + 1)

print("Starting full-scene inference with nodata masking...")
with torch.no_grad():
    for r in range(n_rows):
        if r % 10 == 0:
            print(f"Processing row {r}/{n_rows}...")
            
        for c in range(n_cols):
            y0 = min(r * stride, H - PATCH_SIZE)
            x0 = min(c * stride, W - PATCH_SIZE)
            
            x_patch = L1_norm[:, y0:y0+PATCH_SIZE, x0:x0+PATCH_SIZE]
            x_tensor = torch.from_numpy(x_patch[None, ...]).float().to(device)
            
            y_pred = model(x_tensor).cpu().numpy()[0, 0]
            
            pred_sum[y0:y0+PATCH_SIZE, x0:x0+PATCH_SIZE] += y_pred
            pred_count[y0:y0+PATCH_SIZE, x0:x0+PATCH_SIZE] += 1.0

# Average predictions
pred_norm_full = np.zeros_like(pred_sum)
valid_pred = pred_count > 0
pred_norm_full[valid_pred] = pred_sum[valid_pred] / pred_count[valid_pred]

# De-normalize to Kelvin
pred_kelvin_full = pred_norm_full * (L2_p99 - L2_p1) + L2_p1

# IMPORTANT: Apply nodata mask - set invalid pixels to 0 (nodata)
pred_kelvin_full[~nodata_mask] = 0
print(f"Applied nodata mask: {np.sum(~nodata_mask):,} pixels set to nodata")

# Clip predictions to reasonable range (avoids extreme outliers)
valid_pred_mask = nodata_mask & (pred_kelvin_full > 0)
pred_kelvin_full = np.clip(pred_kelvin_full, L2_p1 - 20, L2_p99 + 20)
pred_kelvin_full[~nodata_mask] = 0  # Re-apply nodata after clipping

# Save TIF
pred_profile = l2_profile.copy()
pred_profile.update(dtype=rasterio.float32, count=1, nodata=0)

with rasterio.open("sol3_prediction.tif", "w", **pred_profile) as dst:
    dst.write(pred_kelvin_full.astype(np.float32), 1)
    
print("Saved: sol3_prediction.tif")


In [ ]:
# Cell 9: Comprehensive Metrics Calculation
print("="*60)
print("COMPREHENSIVE ACCURACY METRICS")
print("="*60)

# Ground truth in Kelvin
true_kelvin = L2
pred_kelvin = pred_kelvin_full

# Create mask for valid pixels (both true and predicted must be valid)
mask = nodata_mask & (pred_kelvin > 0)
n_valid = np.sum(mask)

# Extract valid pixel values
true_vals = true_kelvin[mask]
pred_vals = pred_kelvin[mask]
error = pred_vals - true_vals
abs_error = np.abs(error)

# =========================
# 1. BASIC ERROR METRICS
# =========================
mae = np.mean(abs_error)
rmse = np.sqrt(np.mean(error**2))
max_error = np.max(abs_error)
min_error = np.min(abs_error)

print(f"\n1. BASIC ERROR METRICS:")
print(f"   MAE (Mean Absolute Error):    {mae:.3f} K")
print(f"   RMSE (Root Mean Square Error): {rmse:.3f} K")
print(f"   Max Absolute Error:            {max_error:.3f} K")
print(f"   Min Absolute Error:            {min_error:.3f} K")

# =========================
# 2. BIAS METRICS
# =========================
bias = np.mean(error)
median_error = np.median(error)

print(f"\n2. BIAS METRICS:")
print(f"   Mean Bias (ME):     {bias:+.3f} K {'(over-predicting)' if bias > 0 else '(under-predicting)'}")
print(f"   Median Error:       {median_error:+.3f} K")

# =========================
# 3. CORRELATION METRICS
# =========================
ss_res = np.sum(error**2)
ss_tot = np.sum((true_vals - np.mean(true_vals))**2)
r2 = 1 - (ss_res / ss_tot)
correlation = np.corrcoef(true_vals, pred_vals)[0, 1]

print(f"\n3. CORRELATION METRICS:")
print(f"   R2 Score:           {r2:.5f} ({r2*100:.2f}% variance explained)")
print(f"   Pearson Correlation: {correlation:.5f}")

# =========================
# 4. PERCENTILE ERROR METRICS
# =========================
p50 = np.percentile(abs_error, 50)
p90 = np.percentile(abs_error, 90)
p95 = np.percentile(abs_error, 95)
p99 = np.percentile(abs_error, 99)

print(f"\n4. PERCENTILE ERROR DISTRIBUTION:")
print(f"   50th percentile (median): {p50:.3f} K")
print(f"   90th percentile:          {p90:.3f} K")
print(f"   95th percentile:          {p95:.3f} K")
print(f"   99th percentile:          {p99:.3f} K")

# =========================
# 5. ERROR THRESHOLDS
# =========================
within_1K = np.sum(abs_error <= 1.0) / n_valid * 100
within_2K = np.sum(abs_error <= 2.0) / n_valid * 100
within_5K = np.sum(abs_error <= 5.0) / n_valid * 100

print(f"\n5. ERROR THRESHOLDS (% of pixels within error):")
print(f"   Within +/-1 K:  {within_1K:.2f}%")
print(f"   Within +/-2 K:  {within_2K:.2f}%")
print(f"   Within +/-5 K:  {within_5K:.2f}%")

# =========================
# 6. TEMPERATURE RANGE ANALYSIS
# =========================
print(f"\n6. TEMPERATURE RANGE ANALYSIS:")
print(f"   True temp range:      {true_vals.min():.2f} K to {true_vals.max():.2f} K")
print(f"   Predicted temp range: {pred_vals.min():.2f} K to {pred_vals.max():.2f} K")
print(f"   True mean temp:       {true_vals.mean():.2f} K")
print(f"   Predicted mean temp:  {pred_vals.mean():.2f} K")

# =========================
# 7. SUMMARY SCORE
# =========================
accuracy_score = (r2 * 0.4) + ((1 - mae/10) * 0.3) + (within_2K/100 * 0.3)
print(f"\n7. OVERALL ACCURACY SCORE: {accuracy_score:.4f} (0-1 scale, higher is better)")

print("\n" + "="*60)

# Save metrics to file
with open("sol3_metrics.txt", "w") as f:
    f.write("COMPREHENSIVE ACCURACY METRICS (solAutoencoder3)\n")
    f.write("="*60 + "\n\n")
    f.write(f"MAE: {mae:.3f} K\n")
    f.write(f"RMSE: {rmse:.3f} K\n")
    f.write(f"R2 Score: {r2:.5f}\n")
    f.write(f"Bias (ME): {bias:+.3f} K\n")
    f.write(f"Correlation: {correlation:.5f}\n")
    f.write(f"Max Error: {max_error:.3f} K\n")
    f.write(f"Within +/-1K: {within_1K:.2f}%\n")
    f.write(f"Within +/-2K: {within_2K:.2f}%\n")
    f.write(f"Within +/-5K: {within_5K:.2f}%\n")
    f.write(f"Accuracy Score: {accuracy_score:.4f}\n")
    
print("Saved metrics to sol3_metrics.txt")


In [ ]:
# Cell 10: Generate Comprehensive Comparison Plot
print("Generating comparison plots...")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Row 1: Temperature maps
vmin = np.percentile(true_kelvin[mask], 2)
vmax = np.percentile(true_kelvin[mask], 98)
err_max = np.percentile(np.abs(error), 98)

# True Temperature
im0 = axes[0, 0].imshow(true_kelvin, cmap='inferno', vmin=vmin, vmax=vmax)
axes[0, 0].set_title("True L2 Temperature (K)", fontsize=12)
plt.colorbar(im0, ax=axes[0, 0], label='K')
axes[0, 0].axis('off')

# Predicted Temperature
im1 = axes[0, 1].imshow(pred_kelvin, cmap='inferno', vmin=vmin, vmax=vmax)
axes[0, 1].set_title("Predicted Temperature (K)", fontsize=12)
plt.colorbar(im1, ax=axes[0, 1], label='K')
axes[0, 1].axis('off')

# Error Map
error_full = pred_kelvin - true_kelvin
error_full[~nodata_mask] = 0  # Mask nodata in error map
im2 = axes[0, 2].imshow(error_full, cmap='bwr', vmin=-err_max, vmax=err_max)
axes[0, 2].set_title("Error (Pred - True) (K)", fontsize=12)
plt.colorbar(im2, ax=axes[0, 2], label='K')
axes[0, 2].axis('off')

# Row 2: Statistical plots
# Scatter plot
sample_idx = np.random.choice(len(true_vals), size=min(50000, len(true_vals)), replace=False)
axes[1, 0].scatter(true_vals[sample_idx], pred_vals[sample_idx], alpha=0.1, s=1)
axes[1, 0].plot([true_vals.min(), true_vals.max()], [true_vals.min(), true_vals.max()], 'r--', lw=2, label='Perfect Fit')
axes[1, 0].set_xlabel('True Temperature (K)')
axes[1, 0].set_ylabel('Predicted Temperature (K)')
axes[1, 0].set_title(f'Scatter Plot (R2={r2:.4f})', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Error Histogram
axes[1, 1].hist(error, bins=100, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(x=0, color='r', linestyle='--', label='Zero Error')
axes[1, 1].axvline(x=bias, color='g', linestyle='-', label=f'Bias={bias:.2f}K')
axes[1, 1].set_xlabel('Error (K)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Error Distribution', fontsize=12)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Metrics Summary
metrics_text = f'''
METRICS SUMMARY (sol3)
------------------------
MAE:        {mae:.3f} K
RMSE:       {rmse:.3f} K
R2 Score:   {r2:.5f}
Bias:       {bias:+.3f} K
Correlation:{correlation:.5f}

Within +/-1K: {within_1K:.1f}%
Within +/-2K: {within_2K:.1f}%
Within +/-5K: {within_5K:.1f}%

90th %ile:  {p90:.3f} K
95th %ile:  {p95:.3f} K
'''
axes[1, 2].text(0.1, 0.5, metrics_text, transform=axes[1, 2].transAxes, 
                fontsize=12, verticalalignment='center', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1, 2].axis('off')
axes[1, 2].set_title('Metrics Summary', fontsize=12)

plt.tight_layout()
plt.savefig("sol3_comparison_output.png", dpi=150)
print("Saved comparison plot to sol3_comparison_output.png")
